In [49]:
%reset -f
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import HTML
from pybounds import Simulator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
from double_pendulum import DoublePendulumParameters, DoublePendulumModel, DoublePendulumDrawer

# Set model parameters

In [51]:
dt = 0.05  #  time-step [s]

In [52]:
params = DoublePendulumParameters(
    g=0.0,  # gravity
    k_1=0.0, k_2=1.0,  # torsional stiffness
    c_1=1.0, c_2=1.0,  # torsional damping
    L_1=1.0, L_2=1.0,  # length
    m_1=1.0, m_2=1.0,  # mass
    c_w_1=0.0, c_w_2=0.0,  # wind damping
    J_1=None, J_2=None,  # inertia automatically computed based on cylinder model
)
model = DoublePendulumModel(parameters=params)
model.parameters.__dict__

{'g': 0.0,
 'L_1': 1.0,
 'L_2': 1.0,
 'k_1': 0.0,
 'k_2': 1.0,
 'c_1': 1.0,
 'c_2': 1.0,
 'm_1': 1.0,
 'm_2': 1.0,
 'c_w_1': 2.0,
 'c_w_2': 2.0,
 'J_1': 0.08333333333333333,
 'J_2': 0.08333333333333333}

# Construct simulator object

In [53]:
simulator = Simulator(model.f, model.h,
                      dt=dt,
                      state_names=model.state_names,
                      input_names=model.input_names,
                      measurement_names=model.measurement_names)

# Set inputs

In [54]:
T = 5.0  # total time [s]
tsim = np.arange(start=0.0, stop=T + 1e-3, step=dt)  # time vector [s]

In [55]:
setpoint = {'x': 1.0*np.sin(2*np.pi*(1/T)*tsim),
            'y': 1.0*np.cos(2*np.pi*(1/T)*tsim),
            'theta_1': (np.pi/16) * np.sin(2*np.pi*(3/T)*tsim),
            'theta_2': 0.0 * np.ones_like(tsim),
            }

# Update the simulator set-point
simulator.update_dict(setpoint, name='setpoint')

# Set up model predictive control

In [56]:
# Define cost function
cost = ((simulator.model.x['x'] - simulator.model.tvp['x_set']) ** 2 +
        (simulator.model.x['y'] - simulator.model.tvp['y_set']) ** 2 +
        (simulator.model.x['theta_1'] - simulator.model.tvp['theta_1_set']) ** 2
        )

# Set cost function
simulator.mpc.set_objective(mterm=cost, lterm=cost)

In [57]:
# Set input penalty: make this small for accurate state following
simulator.mpc.set_rterm(x_ddot=1e-6, y_ddot=1e-6, tau_1=1e-6, tau_2=1e12)

# Run simulation in closed-loop

In [58]:
t_sim, x_sim, u_sim, y_sim = simulator.simulate(mpc=True,  return_full_output=True)
sim_data = pd.DataFrame(y_sim)
sim_data.insert(0, 'time', t_sim)
sim_data

,time,theta_1,theta_2,theta_dot_1,theta_dot_2,x,y,x_dot,y_dot,w,zeta,x_ddot,y_ddot,tau_1,tau_2,theta_2_1
0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,24.992698,-1.470647,33.437463,-5.987318e-17,0.000000
1,0.05,0.022478,-0.075995,0.742937,-2.729094,0.031241,0.998162,1.249635,-0.073532,0.0,0.0,24.992698,-1.470647,33.437463,-5.987318e-17,-0.098473
2,0.10,0.064572,-0.224046,0.890962,-3.101288,0.104877,0.992705,1.695806,-0.144720,0.0,0.0,8.923423,-1.423751,21.329746,-5.642690e-17,-0.288618
3,0.15,0.104133,-0.352209,0.736036,-2.112588,0.184601,0.983055,1.493169,-0.241315,0.0,0.0,-4.052740,-1.931894,6.980047,-4.291005e-17,-0.456343
4,0.20,0.135738,-0.432689,0.569883,-1.187403,0.252728,0.968799,1.231894,-0.328908,0.0,0.0,-5.225511,-1.751867,1.904655,-3.765408e-17,-0.568426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,4.80,-0.134645,0.351192,0.540832,-1.820882,-0.249826,0.968590,1.224906,0.319306,0.0,0.0,0.948978,-1.390133,6.320211,-6.039795e-17,0.485836
97,4.85,-0.104878,0.257194,0.640421,-1.924044,-0.186478,0.982613,1.309013,0.241615,0.0,0.0,1.682141,-1.553823,7.454638,-6.644720e-17,0.362072
98,4.90,-0.070467,0.160702,0.725857,-1.919312,-0.120336,0.992396,1.336678,0.149707,0.0,0.0,0.553312,-1.838159,7.054926,-7.081365e-17,0.231169
99,4.95,-0.034795,0.077567,0.709708,-1.427035,-0.058885,0.997752,1.121334,0.064507,0.0,0.0,-4.306896,-1.704004,2.097915,-6.776268e-17,0.112362


# Animate

In [59]:
plt.rcParams['animation.embed_limit'] = 100

In [60]:
drawer = DoublePendulumDrawer(
    sim_data['x'].values,
    sim_data['y'].values,
    sim_data['theta_1'].values,
    sim_data['theta_2'].values,
    L1=model.parameters.L_1,
    L2=model.parameters.L_2,
    seg1_kwargs=dict(color='blue', linewidth=4.0, markersize=7),
    seg2_kwargs=dict(color='red', linewidth=4.0, markersize=7),
    trail1=True, trail2=True, trail_base=True
)

In [61]:
# Make figure
fig, ax = plt.subplots(nrows=1, ncols=1)

def update(frame):
    ax.clear()
    drawer.draw(ax, frame=frame)
    ax.set_aspect('equal')
    ax.axis(drawer.get_axis_bounds())

# Make animation
animation = mpl.animation.FuncAnimation(
    fig, update,
    frames=sim_data.shape[0],
    blit=False,
    interval=int(1000 * dt))

# update(0)
plt.close(fig)

In [62]:
# Display it
HTML(animation.to_jshtml())

In [64]:
# Save it
# from matplotlib.animation import PillowWriter
# save_path = os.path.join(os.path.pardir, 'animation', 'example_model predictive_control.gif')
# animation.save(save_path, PillowWriter(fps=int(1/dt)))